# Create daughter plate

In [1]:
import sys
# sys.path.append('/home/apaulson/repos/data-proc/')
import mysql_cnx as mc
%config Completer.use_jedi=False

In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import text, select, or_, and_

In [3]:
engine.dispose()

NameError: name 'engine' is not defined

In [4]:
# engine=mc.create_mysql_cnx(db='QB3_HiTS_nightly')

engine=mc.create_mysql_cnx(db='QB3_HiTS')

with engine.connect() as conn:
    result=conn.execute(text("select 'hello world'"))
    print(result.all())

[('hello world',)]


In [5]:
# MetaData stores a collection of tables we want to work with
from sqlalchemy import MetaData
metadata = MetaData()

In [6]:
# reflection gets an existing table from the db
from sqlalchemy import Table

inv_plate=Table("INV_PLATE", metadata, autoload_with=engine)
inv_well=Table("INV_WELL", metadata, autoload_with=engine)
inv_comp_lot=Table("INV_COMP_LOT", metadata, autoload_with=engine)

In [7]:
inv_plate_cols=[x.name for x in inv_plate.columns]
inv_plate_cols=[x for x in inv_plate_cols if x != inv_plate.primary_key.columns[0].name]

inv_well_cols=[x.name for x in inv_well.columns]
inv_well_cols=[x for x in inv_well_cols if x != inv_well.primary_key.columns[0].name]

In [8]:
# insert
from sqlalchemy import insert
from datetime import datetime

In [9]:
def stringnow():
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [10]:
inv_plate.primary_key.columns[0].name

'IP_ID'

## lookup plate and well info to copy here

In [39]:
query=select(inv_plate).where(inv_plate.c.IP_GRID_NUM.like("EnamCov%"))
gridnums=pd.read_sql(query, con=engine)
gridnums=gridnums[['IP_GRID_NUM','IP_BARCODE']]
gridnums.columns=['grid_num','barcodes']

In [40]:
# gridnums=pd.read_csv('/Users/apaulson/Downloads/daughter_plates.csv')
# barcodes=barcodes.Grid_num.tolist()

In [41]:
# gridnums=pd.DataFrame(barcodes, columns=['barcodes'])
# print(gridnums.shape)
# gridnums['grid_num']=gridnums.barcodes.str.strip('-Eco1')
gridnums.barcodes=gridnums.barcodes.str.strip()
gridnums.barcodes=gridnums.barcodes.str.replace('MA01','Eco1')
gridnums.shape

(23, 2)

In [42]:
gridnums=gridnums.drop_duplicates()
gridnums.shape

(23, 2)

In [43]:
# check for existing barcodes
query=select(inv_plate.c.IP_BARCODE).where(inv_plate.c.IP_BARCODE.in_(gridnums.barcodes.tolist()))
existing_barcodes=[]
with engine.connect() as conn:
    for row in conn.execute(query):
        existing_barcodes.append(row[0])
len(existing_barcodes)

0

In [44]:
# remove if so
gridnums=gridnums[~gridnums.barcodes.isin(existing_barcodes)]
gridnums.shape

(23, 2)

In [45]:
# double check plate type, should be 1 if possible
clauses = [inv_plate.c.IP_GRID_NUM.like(elem) for elem in gridnums.grid_num]
query = select(inv_plate).filter(and_(
    inv_plate.c.IP_IPT_ID==1, 
    or_(*clauses)))
source_plates=pd.read_sql(query, con=engine)
# source_plates=source_plates[source_plates.IP_BARCODE.str.contains('PT01')]
source_plates.shape

(23, 16)

In [46]:
query = select(inv_well).where(inv_well.c.IW_IP_ID.in_(source_plates.IP_ID.tolist()))
source_wells=pd.read_sql(query, con=engine)
print(len(source_wells)/len(source_plates))
source_wells.head(2)

384.0


,IW_ID,IW_SOURCE_ID,IW_IP_ID,IW_BARCODE,IW_WELL_INDEX,IW_COORDINATES,IW_ICL_ID,IW_VOL_UL,IW_CONC_UM,IW_FINAL_CONC_NM,IW_DILUTION_FACTOR,IW_FREEZETHAW,IW_INS_BY,IW_INS_DATE,IW_CONTAM,IW_NOTES
0,8515222,None,27668,EnamCov01-MA01,1,A01,NaN,NaN,NaN,None,None,None,569,2023-06-20 10:44:10,0,None
1,8515223,None,27668,EnamCov01-MA01,2,A02,NaN,NaN,NaN,None,None,None,569,2023-06-20 10:44:10,0,None


In [47]:
source_plate_dict=dict(zip(source_plates.IP_BARCODE, source_plates.IP_ID))
source_well_dict=dict(zip(source_wells.IW_ID, source_wells.IW_IP_ID.astype(str)+'_'+source_wells.IW_COORDINATES))
dest_plate_dict=dict(zip(gridnums.grid_num, gridnums.barcodes))
source_plate_dict

{'EnamCov01-MA01': 27668,
 'EnamCov02-MA01': 27669,
 'EnamCov03-MA01': 27670,
 'EnamCov04-MA01': 27671,
 'EnamCov05-MA01': 27672,
 'EnamCov06-MA01': 27673,
 'EnamCov07-MA01': 27674,
 'EnamCov08-MA01': 27675,
 'EnamCov09-MA01': 27676,
 'EnamCov10-MA01': 27677,
 'EnamCov11-MA01': 27678,
 'EnamCov12-MA01': 27679,
 'EnamCov13-MA01': 27680,
 'EnamCov14-MA01': 27681,
 'EnamCov15-MA01': 27682,
 'EnamCov16-MA01': 27683,
 'EnamCov17-MA01': 27684,
 'EnamCov18-MA01': 27685,
 'EnamCov19-MA01': 27686,
 'EnamCov20-MA01': 27687,
 'EnamCov21-MA01': 27688,
 'EnamCov22-MA01': 27689,
 'EnamCov23-MA01': 27690}

In [49]:
plates=source_plates.copy()
plates.IP_SOURCE_ID=plates.IP_BARCODE.map(source_plate_dict)
plates=plates.drop(columns='IP_ID')
plates.IP_IPT_ID=9 # 9 is acoustic dispensing plates
plates.IP_BARCODE=plates.IP_GRID_NUM.map(dest_plate_dict)
plates.IP_INS_BY=569
plates.IP_INS_DATE=stringnow()

# plates.IP_NOTES=gridnums.notes.tolist()
plates

,IP_SOURCE_ID,IP_GRID_NUM,IP_IPT_ID,IP_IPF_ID,IP_DESC,IP_BARCODE,IP_STATUS_ID,IP_IV_ID,IP_VEN_PLID,IP_ICO_ID,IP_COLLECTION,IP_NOTES,IP_INS_BY,IP_INS_DATE,IP_SCREENING_CENTER
0,27668,EnamCov01,9,2,None,EnamCov01-Eco1,1,22,1688949-Y40-01,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
1,27669,EnamCov02,9,2,None,EnamCov02-Eco1,1,22,1688949-Y40-02,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
2,27670,EnamCov03,9,2,None,EnamCov03-Eco1,1,22,1688949-Y40-03,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
3,27671,EnamCov04,9,2,None,EnamCov04-Eco1,1,22,1688949-Y40-04,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
4,27672,EnamCov05,9,2,None,EnamCov05-Eco1,1,22,1688949-Y40-05,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
5,27673,EnamCov06,9,2,None,EnamCov06-Eco1,1,22,1688949-Y40-06,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
6,27674,EnamCov07,9,2,None,EnamCov07-Eco1,1,22,1688949-Y40-07,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
7,27675,EnamCov08,9,2,None,EnamCov08-Eco1,1,22,1688949-Y40-08,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
8,27676,EnamCov09,9,2,None,EnamCov09-Eco1,1,22,1688949-Y40-09,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
9,27677,EnamCov10,9,2,None,EnamCov10-Eco1,1,22,1688949-Y40-10,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC


In [50]:
# check for important plate cols
assert set(plates.columns)==set(inv_plate_cols), "Missing info for INV_PLATE table"
assert len(plates[plates.IP_GRID_NUM.isna()])==0, "Missing GridNum"
assert len(plates[plates.IP_BARCODE.isna()])==0, "Missing Barcode"
assert len(plates[plates.IP_GRID_NUM.duplicated()])==0, "Duplicate gridnums"
assert len(plates[plates.IP_BARCODE.duplicated()])==0, "Duplicate barcodes"

In [51]:
# check existing IP_BARCODES
from sqlalchemy import select

stmt=select(inv_plate).where(inv_plate.c.IP_BARCODE.in_(plates.IP_BARCODE.tolist()))
with engine.connect() as conn:
    result=conn.execute(stmt)
    exists=result.rowcount>0

if exists:
    for i, row in enumerate(result):
        print(row)
        ip_id=row[0]
        assert i==0, "Warning: this barcode already exists"
else:
    # stmt=insert(inv_plate).values(plate)    
    # compiled=stmt.compile()
    # with engine.connect() as conn:
    #     result=conn.execute(stmt)
    #     conn.commit()
    # ip_id=result.inserted_primary_key
    ip_id=np.nan
    print('OK')

OK


In [52]:
ip_id

nan

In [53]:
import sqlalchemy
if isinstance(ip_id,sqlalchemy.engine.row.Row):
    ip_id=ip_id[0]
ip_id

nan

In [54]:
# # insert.. careful!!
# plates.to_sql(name='INV_PLATE', con=engine, if_exists = 'append', index=False)

23

In [55]:
# get IP_IDs
ip_ids=pd.read_sql(stmt, con=engine)
ip_ids

,IP_ID,IP_SOURCE_ID,IP_GRID_NUM,IP_IPT_ID,IP_IPF_ID,IP_DESC,IP_BARCODE,IP_STATUS_ID,IP_IV_ID,IP_VEN_PLID,IP_ICO_ID,IP_COLLECTION,IP_NOTES,IP_INS_BY,IP_INS_DATE,IP_SCREENING_CENTER
0,27877,27668,EnamCov01,9,2,None,EnamCov01-Eco1,1,22,1688949-Y40-01,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
1,27878,27669,EnamCov02,9,2,None,EnamCov02-Eco1,1,22,1688949-Y40-02,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
2,27879,27670,EnamCov03,9,2,None,EnamCov03-Eco1,1,22,1688949-Y40-03,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
3,27880,27671,EnamCov04,9,2,None,EnamCov04-Eco1,1,22,1688949-Y40-04,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
4,27881,27672,EnamCov05,9,2,None,EnamCov05-Eco1,1,22,1688949-Y40-05,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
5,27882,27673,EnamCov06,9,2,None,EnamCov06-Eco1,1,22,1688949-Y40-06,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
6,27883,27674,EnamCov07,9,2,None,EnamCov07-Eco1,1,22,1688949-Y40-07,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
7,27884,27675,EnamCov08,9,2,None,EnamCov08-Eco1,1,22,1688949-Y40-08,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
8,27885,27676,EnamCov09,9,2,None,EnamCov09-Eco1,1,22,1688949-Y40-09,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC
9,27886,27677,EnamCov10,9,2,None,EnamCov10-Eco1,1,22,1688949-Y40-10,241,Enamine Covalent,None,569,2023-11-01 16:45:27,SMDC


## edit well info here via plate map file

In [56]:
# wells
replace_ipids=dict(zip(ip_ids.IP_SOURCE_ID, ip_ids.IP_ID))

In [57]:
wells=source_wells.copy()
wells.IW_SOURCE_ID=wells.IW_ID
wells=wells.drop(columns='IW_ID')
wells.IW_IP_ID=wells.IW_IP_ID.map(replace_ipids)
wells.IW_INS_BY=569
wells.IW_INS_DATE=stringnow()
wells.head(2)

,IW_SOURCE_ID,IW_IP_ID,IW_BARCODE,IW_WELL_INDEX,IW_COORDINATES,IW_ICL_ID,IW_VOL_UL,IW_CONC_UM,IW_FINAL_CONC_NM,IW_DILUTION_FACTOR,IW_FREEZETHAW,IW_INS_BY,IW_INS_DATE,IW_CONTAM,IW_NOTES
0,8515222,27877,EnamCov01-MA01,1,A01,NaN,NaN,NaN,None,None,None,569,2023-11-01 16:46:13,0,None
1,8515223,27877,EnamCov01-MA01,2,A02,NaN,NaN,NaN,None,None,None,569,2023-11-01 16:46:13,0,None


### add volume and concentration here

In [58]:
wells.IW_CONC_UM.unique()

array([   nan, 20000.])

In [59]:
vol_ul=6
conc_um=20000

In [60]:
wells.loc[~wells.IW_ICL_ID.isna(), 'IW_VOL_UL']=vol_ul
wells.loc[~wells.IW_ICL_ID.isna(), 'IW_CONC_UM']=conc_um
wells

,IW_SOURCE_ID,IW_IP_ID,IW_BARCODE,IW_WELL_INDEX,IW_COORDINATES,IW_ICL_ID,IW_VOL_UL,IW_CONC_UM,IW_FINAL_CONC_NM,IW_DILUTION_FACTOR,IW_FREEZETHAW,IW_INS_BY,IW_INS_DATE,IW_CONTAM,IW_NOTES
0,8515222,27877,EnamCov01-MA01,1,A01,NaN,NaN,NaN,None,None,None,569,2023-11-01 16:46:13,0,None
1,8515223,27877,EnamCov01-MA01,2,A02,NaN,NaN,NaN,None,None,None,569,2023-11-01 16:46:13,0,None
2,8515224,27877,EnamCov01-MA01,3,A03,1278555.0,6.0,20000.0,None,None,None,569,2023-11-01 16:46:13,0,None
3,8515225,27877,EnamCov01-MA01,4,A04,1278571.0,6.0,20000.0,None,None,None,569,2023-11-01 16:46:13,0,None
4,8515226,27877,EnamCov01-MA01,5,A05,1278587.0,6.0,20000.0,None,None,None,569,2023-11-01 16:46:13,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8827,8524049,27899,EnamCov23-MA01,380,P20,1285882.0,6.0,20000.0,None,None,None,569,2023-11-01 16:46:13,0,None
8828,8524050,27899,EnamCov23-MA01,381,P21,1285898.0,6.0,20000.0,None,None,None,569,2023-11-01 16:46:13,0,None
8829,8524051,27899,EnamCov23-MA01,382,P22,1285914.0,6.0,20000.0,None,None,None,569,2023-11-01 16:46:13,0,None
8830,8524052,27899,EnamCov23-MA01,383,P23,NaN,NaN,NaN,None,None,None,569,2023-11-01 16:46:13,0,None


In [61]:
# check IP_ID doesn't exist in INV_WELL yet
stmt=select(inv_well).where(inv_well.c.IW_IP_ID.in_(tuple(wells.IW_IP_ID.unique())))
with engine.connect() as conn:
    result=conn.execute(stmt)
    assert result.rowcount==0, "Wells exist for these plates"

In [62]:
wells.shape, len(wells)/384

((8832, 15), 23.0)

In [64]:
# insert... careful here
wells.to_sql(name='INV_WELL', con=engine, if_exists = 'append', index=False)

8832

In [65]:
engine.dispose()

In [66]:
existing_barcodes

[]

In [67]:
gridnums.barcodes.tolist()

['EnamCov01-Eco1',
 'EnamCov02-Eco1',
 'EnamCov03-Eco1',
 'EnamCov04-Eco1',
 'EnamCov05-Eco1',
 'EnamCov06-Eco1',
 'EnamCov07-Eco1',
 'EnamCov08-Eco1',
 'EnamCov09-Eco1',
 'EnamCov10-Eco1',
 'EnamCov11-Eco1',
 'EnamCov12-Eco1',
 'EnamCov13-Eco1',
 'EnamCov14-Eco1',
 'EnamCov15-Eco1',
 'EnamCov16-Eco1',
 'EnamCov17-Eco1',
 'EnamCov18-Eco1',
 'EnamCov19-Eco1',
 'EnamCov20-Eco1',
 'EnamCov21-Eco1',
 'EnamCov22-Eco1',
 'EnamCov23-Eco1']